In [1]:
from openff.qcsubmit.common_structures import QCSpec, SCFProperties
from openff.qcsubmit.factories import BasicDatasetFactory
import numpy as np
from qcelemental.models.results import WavefunctionProtocolEnum
from qcportal.models.common_models import DriverEnum
from openeye import oechem

In [2]:
factory = BasicDatasetFactory(driver=DriverEnum.gradient,
                              qc_specifications={'spice_default': QCSpec(method='wb97m-d3bj', 
                                                                   basis='def2-tzvppd', 
                                                                   program='psi4', 
                                                                   spec_name='spice_default', 
                                                                   spec_description='SPICE quantum chemistry specification',
                                                                   store_wavefunction=WavefunctionProtocolEnum.orbitals_and_eigenvalues, 
                                                                   implicit_solvent=None, 
                                                                   maxiter=200, 
                                                                   scf_properties=[SCFProperties.Dipole, SCFProperties.Quadrupole, SCFProperties.WibergLowdinIndices, 
                                                                                   SCFProperties.MayerIndices, SCFProperties.MBISCharges],
                                                                   keywords={'wcombine': False})},
                             store_wavefunction=WavefunctionProtocolEnum.orbitals_and_eigenvalues)
factory.dict()

{'qc_specifications': {'spice_default': {'method': 'wb97m-d3bj',
   'basis': 'def2-tzvppd',
   'program': 'psi4',
   'spec_name': 'spice_default',
   'spec_description': 'SPICE quantum chemistry specification',
   'store_wavefunction': 'orbitals_and_eigenvalues',
   'implicit_solvent': None,
   'maxiter': 200,
   'scf_properties': ['dipole',
    'quadrupole',
    'wiberg_lowdin_indices',
    'mayer_indices',
    'mbis_charges'],
   'keywords': {'wcombine': False}}},
 'driver': 'gradient',
 'priority': 'normal',
 'dataset_tags': ['openff'],
 'compute_tag': 'openff',
 'type': 'BasicDatasetFactory',
 'workflow': []}

In [3]:
# mock a dataset with dummy information for now
dataset = factory.create_dataset(dataset_name="SPICE Dipeptides Single Points Dataset v1.1", molecules="dipeptides.hdf5", tagline="QM Dataset for ML", description="Data source: https://github.com/openmm/qmdataset/tree/main/dipeptides")

Preparation                   : 100%|█████████| 677/677 [03:26<00:00,  3.28it/s]


In [4]:
confs = np.array([len(mol.conformers) for mol in dataset.molecules])

print("Number of unique molecules       ", dataset.n_molecules)
print("Number of filtered molecules     ", dataset.n_filtered)
print("Number of conformers             ", dataset.n_records)
print("Number of conformers min mean max",
      confs.min(), "{:6.2f}".format(confs.mean()), confs.max())

masses = []
for molecule in dataset.molecules:
    oemol = molecule.to_openeye()
    mass = oechem.OECalculateMolecularWeight(oemol)
    masses.append(mass)

print(f'Mean molecular weight: {np.mean(np.array(masses)):.2f}')
print(f'Max molecular weight: {np.max(np.array(masses)):.2f}')
print("Charges:", sorted(set(m.total_charge/m.total_charge.unit for m in dataset.molecules)))

Number of unique molecules        677
Number of filtered molecules      0
Number of conformers              33850
Number of conformers min mean max 50  50.00 50
Mean molecular weight: 313.72
Max molecular weight: 445.51
Charges: [-2.0, -1.0, 0.0, 1.0, 2.0]


In [5]:
dataset.visualize("dipeptides.pdf")

In [6]:
dataset.molecules_to_file("dipeptides.smi", "smi")

In [7]:
dataset.metadata.submitter = 'Josh Horton, Pavan Behara, David Dotson'

In [8]:
dataset.metadata.long_description_url = 'https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/2021-11-08-QMDataset-Dipeptide-single-points'

In [9]:
dataset.metadata

Metadata(submitter='Josh Horton, Pavan Behara, David Dotson', creation_date=datetime.date(2021, 12, 7), collection_type='DataSet', dataset_name='SPICE Dipeptides Single Points Dataset v1.1', short_description='QM Dataset for ML', long_description_url=HttpUrl('https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/2021-11-08-QMDataset-Dipeptide-single-points', scheme='https', host='github.com', tld='com', host_type='domain', path='/openforcefield/qca-dataset-submission/tree/master/submissions/2021-11-08-QMDataset-Dipeptide-single-points'), long_description='Data source: https://github.com/openmm/qmdataset/tree/main/dipeptides', elements={'S', 'N', 'H', 'O', 'C'})

In [10]:
dataset.export_dataset("dataset.json.bz2")